# Lab Assignment 3: How to Load, Convert, and Write JSON Files in Python
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

## Problem 0
Import the following libraries:

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks

## Problem 1 
JSON and CSV are both text-based formats for the storage of data. It's possible to open either one in a plain text editor. Given this similarity, why does a CSV file usually take less memory than a JSON formatted file for the same data? Under what conditions could a JSON file be smaller in memory than a CSV file for the same data? (2 points)

CSV files usually take less memory than JSON files for the same data because CSV files are a flat, tabular format without any structural metadata, while JSON files include structural elements. This additional metadata in JSON increases the file size.

A JSON file might be smaller if it contains deeply nested structures or repeated field names that benefit from JSON's hierarchical storage. JSON format can "compress" data more efficiently than a CSV file, which would need to repeat the field names for each record.


## Problem 2
NASA has a dataset of all meteorites that have fallen to Earth between the years A.D. 860 and 2013. The data contain the name of each meteorite, along with the coordinates of the place where the meteorite hit, the mass of the meteorite, and the date of the collison. The data is stored as a JSON here: https://data.nasa.gov/resource/y77d-th95.json

Look at the data in your web-browser and explain which strategy for loading the JSON into Python makes the most sense and why. 

Then write and run the code that will work for loading the data into Python. (2 points)

In [2]:
url = 'https://data.nasa.gov/resource/y77d-th95.json'
response = requests.get(url)
response.raise_for_status()
data = json.loads(response.text)
df = pd.json_normalize(data)

print(df.head())

       name   id nametype     recclass    mass  fall                     year  \
0    Aachen    1    Valid           L5      21  Fell  1880-01-01T00:00:00.000   
1    Aarhus    2    Valid           H6     720  Fell  1951-01-01T00:00:00.000   
2      Abee    6    Valid          EH4  107000  Fell  1952-01-01T00:00:00.000   
3  Acapulco   10    Valid  Acapulcoite    1914  Fell  1976-01-01T00:00:00.000   
4   Achiras  370    Valid           L6     780  Fell  1902-01-01T00:00:00.000   

       reclat      reclong geolocation.type geolocation.coordinates  \
0   50.775000     6.083330            Point       [6.08333, 50.775]   
1   56.183330    10.233330            Point    [10.23333, 56.18333]   
2   54.216670  -113.000000            Point        [-113, 54.21667]   
3   16.883330   -99.900000            Point       [-99.9, 16.88333]   
4  -33.166670   -64.950000            Point     [-64.95, -33.16667]   

  :@computed_region_cbhk_fwbd :@computed_region_nnqa_25f4  
0                         

The strategy that made the most sense was to use requests.get to fetch the raw data, json.loads to convert it into a list, and pd.json_normalize to store each feature as a separate column. This approach was necessary due to the nested structure of the data

## Problem 3
The textbook chapter for this module shows, as an example, how to pull data in JSON format from Reddit's top 25 posts on [/r/popular](https://www.reddit.com/r/popular/top/). The steps outlined there pull all of the features in the data into the dataframe, resulting in a dataframe with 172 columns. 

If we only wanted a few features, then looping across elements of the JSON list itself and extracting only the data we want may be a more efficient approach.

Use looping - and not `pd.read_json()` or `pd.json_normalize()` - to create a dataframe with 25 rows (one for each of the top 25 posts), and only columns for `subreddit`, `title`, `ups`, and `created_utc`. The JSON file exists at http://www.reddit.com/r/popular/top.json, and don't forget to specify `headers = {'User-agent': 'DS6001'}` within `requests.get()`. (3 points)

In [4]:
url = 'https://www.reddit.com/r/popular/top.json'
headers = {'User-agent': 'DS6001'}
response = requests.get(url, headers=headers)
response.raise_for_status()

In [5]:
data = json.loads(response.text)
subreddits = [post['data']['subreddit'] for post in data['data']['children']]
titles = [post['data']['title'] for post in data['data']['children']]
ups = [post['data']['ups'] for post in data['data']['children']]
created_utcs = [post['data']['created_utc'] for post in data['data']['children']]

In [6]:
df = pd.DataFrame({
    'subreddit': subreddits,
    'title': titles,
    'ups': ups,
    'created_utc': created_utcs
})
df.head()

,subreddit,title,ups,created_utc
0,Damnthatsinteresting,Mosquito coil holder made using a 3D printing ...,58863,1.719775e+09
1,interestingasfuck,The Chinese Tianlong-3 Rocket Accidentally Lau...,56122,1.719747e+09
2,mildlyinfuriating,To the guy who is mildly infuriated by their n...,50667,1.719724e+09
3,MadeMeSmile,The hug.... wow,44811,1.719737e+09
4,interestingasfuck,This 9 year old girl dodges and manages to esc...,42909,1.719749e+09


## Problem 4
The NBA has saved data on all 30 teams' shooting statistics for the 2014-2015 season here: https://stats.nba.com/js/data/sportvu/2015/shootingTeamData.json. Take a moment and look at this JSON file in your web browser. The structure of this particular JSON is complicated, but see if you can find the team-by-team data. In this problem our goal is to use `pd.json_normalize()` to get the data into a dataframe. The following questions will guide you towards this goal.

### Part a
Download the raw text of the NBA JSON file and register it as JSON formatted data in Python's memory. (2 points)

### Part b
Describe, in words, the path that leads to the team-by-team data. (2 points)

### Part c
Use the `pd.json_normalize()` function to pull the team-by-team data into a dataframe. This is going to be tricky. You will need to use indexing on the JSON data as well as the `record_path` parameter. 

If you are successful, you will have a dataframe with 30 rows and 33 columns. The first row will refer to the Golden State Warriors, the second row will refer to the San Antonio Spurs, and the third row will refer to the Cleveland Cavaliers. The columns will only be named 0, 1, 2, ... at this point. (4 points)

### Part d
Find the path that leads to the headers (the column names), and extract these names as a list. Then set the `.columns` attribute of the dataframe you created in part c equal to this list. The result should be that the dataframe now has the correct column names. (3 points)

In [7]:
url = 'https://stats.nba.com/js/data/sportvu/2015/shootingTeamData.json'
headers = {'User-agent': 'DS6001'}
response = requests.get(url, headers=headers)
response.raise_for_status()
nba_json = json.loads(response.text)

The path to the team-by-team data is resultSets, the 0th index, and rowSet.

In [9]:
nba_df = pd.json_normalize(nba_json, record_path = ["resultSets", "rowSet"])
nba_df

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,1610612744,Golden State,Warriors,GSW,,82,48.7,114.9,14.9,0.498,...,0.478,21.2,42.5,0.497,2.3,6.3,0.363,10.8,25.3,0.429
1,1610612759,San Antonio,Spurs,SAS,,82,48.3,103.5,14.8,0.481,...,0.506,18.3,39.8,0.460,0.9,2.6,0.341,6.1,15.9,0.381
2,1610612739,Cleveland,Cavaliers,CLE,,82,48.7,104.3,16.9,0.481,...,0.473,18.2,40.7,0.447,1.7,5.7,0.299,9.0,23.9,0.378
3,1610612746,Los Angeles,Clippers,LAC,,82,48.6,104.5,15.0,0.497,...,0.480,18.9,42.0,0.450,2.0,6.0,0.334,7.7,20.8,0.373
4,1610612760,Oklahoma City,Thunder,OKC,,82,48.6,110.2,16.1,0.480,...,0.497,17.5,38.7,0.451,1.6,5.1,0.321,6.6,18.6,0.356
5,1610612737,Atlanta,Hawks,ATL,,82,48.6,102.8,19.0,0.463,...,0.483,19.4,44.6,0.435,1.0,3.1,0.311,9.0,25.3,0.355
6,1610612745,Houston,Rockets,HOU,,82,48.6,106.5,17.2,0.433,...,0.472,15.5,36.4,0.426,2.3,7.4,0.318,8.4,23.5,0.355
7,1610612757,Portland,Trail Blazers,POR,,82,48.5,105.1,17.5,0.441,...,0.447,18.0,39.8,0.453,1.7,5.9,0.295,8.8,22.6,0.389
8,1610612758,Sacramento,Kings,SAC,,81,48.4,106.7,18.7,0.452,...,0.473,18.1,39.7,0.454,0.9,3.1,0.276,7.2,19.4,0.372
9,1610612764,Washington,Wizards,WAS,,82,48.5,104.1,15.4,0.480,...,0.483,19.5,44.3,0.439,0.7,2.7,0.254,8.0,21.5,0.371


In [10]:
column_names = nba_json['resultSets'][0]['headers']
nba_df.columns = column_names
print(nba_df.head())

      TEAM_ID      TEAM_CITY  TEAM_NAME TEAM_ABBREVIATION TEAM_CODE  GP   MIN  \
0  1610612744   Golden State   Warriors               GSW            82  48.7   
1  1610612759    San Antonio      Spurs               SAS            82  48.3   
2  1610612739      Cleveland  Cavaliers               CLE            82  48.7   
3  1610612746    Los Angeles   Clippers               LAC            82  48.6   
4  1610612760  Oklahoma City    Thunder               OKC            82  48.6   

     PTS  PTS_DRIVE  FGP_DRIVE  ...   CFGP  UFGM  UFGA   UFGP  CFG3M  CFG3A  \
0  114.9       14.9      0.498  ...  0.478  21.2  42.5  0.497    2.3    6.3   
1  103.5       14.8      0.481  ...  0.506  18.3  39.8  0.460    0.9    2.6   
2  104.3       16.9      0.481  ...  0.473  18.2  40.7  0.447    1.7    5.7   
3  104.5       15.0      0.497  ...  0.480  18.9  42.0  0.450    2.0    6.0   
4  110.2       16.1      0.480  ...  0.497  17.5  38.7  0.451    1.6    5.1   

   CFG3P  UFG3M  UFG3A  UFG3P  
0  0.3

## Problem 5
Save the NBA dataframe you extracted in problem 4 as a JSON-formatted text file on your local machine. Format the JSON so that it is organized as dictionary with three lists: `columns` lists the column names, `index` lists the row names, and `data` is a list-of-lists of data points, one list for each row. (Hint: this is possible with one line of code) (2 points)

In [11]:
nba_df.to_json("nba.json", orient="split")
